### Find the maximum impacts for individual, local, synergistic and global nudges

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import json

import numpy as np
from scipy import stats

import probability_distributions
import maximum_nudges
import evolutionary_algorithms as ea
import maximum_synergistic_nudge_evolutionary as synergistic_nudge_ev
import maximum_nudges_evolutionary as ev_max_nudges
import nudge_non_causal
import get_data

### Notebook level constants

In [2]:
NUDGE_SIZE = 0.01
PATH_TO_DISTRIBUTIONS = "/home/joboti/azumi_derkjan/master_thesis/code/"

### Find the maximum impact of a nudge given its type the input and conditional output and nudge size

The nudge sizes for individual and global are divided by 2 on purpose, since in their definitions its the 
amount subtracted AND added. 

In [3]:
def get_max_nudge_impacts(input_dists, cond_output_dists, nudge_size, nudge_type, 
                          backup_filename, parameters):
    """
    Find the maximum impact of the nudges
    
    Parameters:
    ----------
    input_dists: list of nd-arrays representing probability distributions
    cond_output_dists: list of nd-arrays 
        Representing output distributions (the last axis) conditioned on the input 
        distributions
    nudge_size: positive float
    nudge_type: string
        One of the following: {"individual", "local", "synergistic", "global"}
    filename_to_save: string, 
        Should be a valid path
    parameters: a dict,
        The parameters used to find the maximum nudge
    
    """
    impacts = []
    for input_dist, cond_output in zip(input_dists, cond_output_dists):
        print("count {}".format(len(impacts)))
        if nudge_type == "individual":
            impact = maximum_nudges.find_max_impact_individual_nudge_exactly(
                input_dist, cond_output, nudge_size/2.0, True
            )
        elif nudge_type == "local":
            max_local_nudge = ev_max_nudges.find_maximum_local_nudge(
                input_dist, cond_output, nudge_size, parameters, verbose=False
            )
            impact = max_local_nudge.score
            individual_nudges = max_local_nudge.individual_nudges
            nudge_vectors = [
                weight*nudge.genes 
                for nudge, weight in zip(individual_nudges, max_local_nudge.weights)
            ]
            new_dist = nudge_non_causal.nudge_local(
                input_dist, [0, 1], 0.01, nudge_vectors, True
            )
            l1_norm_to_old_distribution = np.sum(np.absolute(input_dist-new_dist))
            if abs(l1_norm_to_old_distribution-NUDGE_SIZE) > 10**(-7):
                print("WARNING size of nudge {}".format(l1_norm_to_old_distribution))
        elif nudge_type == "synergistic":
            max_synergistic_nudge = synergistic_nudge_ev.find_synergistic_nudge_with_max_impact(
                input_dist, cond_output, nudge_size, parameters
            )
            impact = max_synergistic_nudge.score
            new_distribution = max_synergistic_nudge.new_distribution
            l1_norm_to_old_distribution = np.sum(np.absolute(
                input_dist-max_synergistic_nudge.new_distribution
            ))
            if abs(l1_norm_to_old_distribution-NUDGE_SIZE) > 10**(-7):
                print("WARNING size of nudge {}".format(l1_norm_to_old_distribution))
        elif nudge_type == "global":
            _, _, max_global_nudge_impact = maximum_nudges.find_max_control_impact(
                input_dist, cond_output, nudge_size/2.0
            )
            impact = max_global_nudge_impact
        else:
            raise ValueError("provide a valid nudge type")
            
        print("the max nudge impact {}".format(impact))
        impacts.append(impact)
        with open(backup_filename, 'w') as f:
            json.dump(impacts, f, indent=4)
        
    return impacts
    

### Generate the data and run the experiments

#### First for system distributions with limited entropy

In [4]:
PERCENTAGE_MAX_ENTROPY_SIZE = 75
NUMBER_OF_VARS = 3
NUMBER_OF_STATES = 3
FILENAME_FORMAT_INPUT = "input_dist_{}.npy"
FILENAME_FORMAT_OUTPUT = "cond_output_dist_{}.npy"
DIST_START = 100
DIST_END = 200
NUDGE_TYPE = "local"

distribution_numbers = list(range(DIST_START, DIST_END, 1))
if NUDGE_TYPE == "global" or NUDGE_TYPE =="individual":
    parameter_file = None
elif NUDGE_TYPE == "local":
    parameter_folder = "local_nudge_parameters/"
    parameter_file_name = "entropy_percentage{}_{}vars_{}states_dist_{}_{}.json".format(
        PERCENTAGE_MAX_ENTROPY_SIZE, NUMBER_OF_VARS, NUMBER_OF_STATES, DIST_START, DIST_END
    )    
    parameter_file = parameter_folder + parameter_file_name 
elif NUDGE_TYPE == "synergistic":
    parameter_folder = "synergistic_nudge_parameters/"
    parameter_file_name = "entropy_percentage{}_{}vars_{}states_dist_{}_{}.json".format(
        PERCENTAGE_MAX_ENTROPY_SIZE, NUMBER_OF_VARS, NUMBER_OF_STATES, DIST_START, DIST_END
    )
    parameter_file = parameter_folder + parameter_file_name 
else:
    raise ValueError("provide a valid nudge type")
    
if parameter_file is None:
    parameters = None
else:
    with open(parameter_file) as f:
        parameters = json.load(f)

backup_filename = ("data_experiments/" + 
    "backup_impacts_{}var_{}states_{}entropy_{}_nudge_dists{}-{}.json".format(
        NUMBER_OF_VARS, NUMBER_OF_STATES, PERCENTAGE_MAX_ENTROPY_SIZE, NUDGE_TYPE, DIST_START, DIST_END 
    )
)
path_to_limited_entropy_system_dists = (
    PATH_TO_DISTRIBUTIONS + "system_distributions/" 
    + "limited_entropy/"
)

input_dists = get_data.get_system_distributions_limited_entropy(
    path_to_limited_entropy_system_dists, PERCENTAGE_MAX_ENTROPY_SIZE,
    NUMBER_OF_VARS, NUMBER_OF_STATES, FILENAME_FORMAT_INPUT, 
    distribution_numbers
)
output_dists = get_data.get_system_distributions_limited_entropy(
    path_to_limited_entropy_system_dists, PERCENTAGE_MAX_ENTROPY_SIZE,
    NUMBER_OF_VARS, NUMBER_OF_STATES, FILENAME_FORMAT_OUTPUT, 
    distribution_numbers
)

impacts = get_max_nudge_impacts(
    input_dists, output_dists, NUDGE_SIZE, NUDGE_TYPE, 
    backup_filename, parameters
)
print("mean impact {}".format(np.mean(impacts)))
print(impacts)

filename_to_save_impacts =  "impacts_{}var_{}states_{}entropy_{}_nudge_dists{}-{}.json".format(
    NUMBER_OF_VARS, NUMBER_OF_STATES, PERCENTAGE_MAX_ENTROPY_SIZE, NUDGE_TYPE, DIST_START, DIST_END 
)
with open("data_experiments/" + filename_to_save_impacts, 'w') as f:
    json.dump(impacts, f)
    
    

count 0
the max nudge impact -0.00897613737155
count 1
WARNING size of nudge 0.00936122726679
the max nudge impact -0.00780824712274
count 2
the max nudge impact -0.00659679264115
count 3
WARNING size of nudge 0.00983393947062
the max nudge impact -0.00770045705801
count 4
the max nudge impact -0.00534779398813
count 5
the max nudge impact -0.00532817585518
count 6
the max nudge impact -0.00611642370847
count 7
the max nudge impact -0.00813577866471
count 8
the max nudge impact -0.00436711018742
count 9
the max nudge impact -0.00477879523788
count 10
the max nudge impact -0.00972670165358
count 11
the max nudge impact -0.00811216680528
count 12
WARNING size of nudge 0.0098422124708
the max nudge impact -0.00711442635333
count 13
the max nudge impact -0.00955064048068
count 14
the max nudge impact -0.00617982989014
count 15
the max nudge impact -0.00745360163724
count 16
the max nudge impact -0.00803330220794
count 17
WARNING size of nudge 0.00960028055265
the max nudge impact -0.008893

#### First generate a generic input distribution

In [ ]:
#distribution parameters
input_variables = 2
number_of_states = 5

#generate both input and conditional output with Dirichlet weights
distribution_shape = [number_of_states]*input_variables
total_number_of_states = reduce(lambda x,y: x*y, distribution_shape)
input_dist = np.random.dirichlet([1]*total_number_of_states)
input_dist = np.reshape(input_dist, distribution_shape)
cond_shape = [number_of_states]*(input_variables+1)
cond_output = [
    probability_distributions.compute_joint_uniform_random((number_of_states,))
    for i in range(number_of_states**(input_variables))
]
cond_output = np.array(cond_output)
cond_output = np.reshape(cond_output, cond_shape)


#### load the generated input and conditional output distribution

In [ ]:
PATH = "/home/joboti/azumi_derkjan/master_thesis/code/"


def generate_distributions(path_to_files, file_format, number_of_distributions):
    for i in range(number_of_distributions):
        file_name = path_to_files + file_format.format(i)
        with open(file_name, 'rb') as f:
            yield np.load(f)
            
def generate_input_and_conditional_output(input_type, parameters, cond_output_type="dirichlet"):
    """
    
    Parameters:
    ----------
    input_type: string in set {"dirichlet", "entropy_0.5", "entropy_0.75"}
    parameters: dict
    cond_output: 
    
    Returns: a dict with keys:
    -------
    number_of_var: a number
    number_of_states: a number
    input_dist: nd-array
    cond_output: nd-array
    
    """
    if input_type == "dirichlet":
        input_dirichlet_path = PATH + INPUT_FOLDER + DIRICHLET_FOLDER_INPUT
    elif input_type == "entropy_0.75":
        input_dirichlet_path = PATH + INPUT_FOLDER + ENTROPY_MEDIUM_FOLDER_INPUT
    elif input_type == "entropy_0.5":
        input_dirichlet_path = PATH + INPUT_FOLDER + ENTROPY_LOW_FOLDER_INPUT
    else:
        raise ValueError("supply valid input distribution type")

    cond_output_dirichlet_path = PATH + COND_OUTPUT_FOLDER + DIRICHLET_FOLDER_COND_OUTPUT

    min_inputs = parameters["min_number_inputs"]
    max_inputs = parameters["max_number_inputs"]
    number_of_states = parameters["number_of_states"]
    for number_of_var in range(min_inputs, max_inputs, 1):
        path_to_input_files = (
            input_dirichlet_path 
            + FOLDER_FORMAT_INPUT.format(number_of_var, number_of_states)
        )
        path_to_cond_output_files = (
            cond_output_dirichlet_path 
            + FOLDER_FORMAT_CONDITIONAL.format(number_of_var, number_of_states)
        )
        input_generator = generate_distributions(
            path_to_input_files, FILE_FORMAT_INPUT, 
            parameters["number_of_distributions"]
        )
        cond_output_generator = generate_distributions(
            path_to_cond_output_files, FILE_FORMAT_COND_OUTPUT, 
            parameters["number_of_distributions"]
        )
        input_shape = [number_of_states]*number_of_var
        cond_output_shape = [number_of_states]*(number_of_var+1)
        for sample in range(parameters["number_of_distributions"]):
            input_dist = next(input_generator)
            input_dist = np.reshape(input_dist, input_shape)
            cond_output = next(cond_output_generator)
            cond_output = np.reshape(cond_output, cond_output_shape)
            yield {
                "number_of_vars": number_of_var,
                "number_of_states": parameters["number_of_states"],
                "input_dist": input_dist,
                "cond_output": cond_output
            }
            